In [64]:
import pandas as pd
df = pd.read_excel('D:\Fin_websites\\acuite_only.xlsx')
company_names = list(df['Company Name'])
company_names = list(set(company_names))
cnt = 0

In [65]:
fin_keys = []
for i in company_names:
    fin_key = ''
    txt_split = i.split(' ')
    for j in range(len(txt_split)):
        if (txt_split[j] == 'PRIVATE' or txt_split[j] == 'LIMITED'):
            txt_split = txt_split[:j]
            break
    for i in range(len(txt_split)):
        if(txt_split[i] == '&'):
            txt_split[i] = 'and'
    if (len(txt_split) >= 4):
        fin_key = '+'.join(txt_split[:4]).lower()
    else:
        fin_key = '+'.join(txt_split).lower()
    fin_keys.append(fin_key)

In [66]:
import requests
import json
def get_name(name):
    url_ans = requests.get('https://connect.acuite.in/search/result?query=' + name)
    json_string = url_ans.text
    data = json.loads(json_string)
    first_company_name = ''
    if data and 'Company_Name' in data[0]:
        first_company_name = data[0]['Company_Name']
    else:
        print("No company name found in the JSON data.")
    return first_company_name

In [70]:
import requests
from bs4 import BeautifulSoup

# First, we'll make a GET request to fetch the CSRF token
not_happen = []
def get_date(company_name):
    session = requests.Session()
    response = session.get("https://connect.acuite.in/liveratings?page=1")
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the CSRF token in the HTML
    csrf_token = soup.find('meta', {'name': 'csrf-token'})['content']

    # Now, let's make the POST request
    url = "https://connect.acuite.in/rating-search-data"

    payload = {
        "search_company_name": company_name
    }

    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Host": "connect.acuite.in",
        "Origin": "https://connect.acuite.in",
        "Referer": "https://connect.acuite.in/liveratings?page=1",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "X-CSRF-TOKEN": csrf_token,  # Use the fetched CSRF token
        "X-Requested-With": "XMLHttpRequest",
        "sec-ch-ua": '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"'
    }

    response = session.post(url, data=payload, headers=headers)
    html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')
    first_date = soup.select_one('tbody.rating-body tr th.td-date')

    if first_date:
        print("First date:", first_date.text.strip())
    else:
        first_date = ''
    
    return str(first_date.text.strip())



In [71]:
import requests
from bs4 import BeautifulSoup
import pdfkit
import os

def html_to_pdf(url, output_file):
    try:
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        response = requests.get(url)
        response.raise_for_status()
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        
    
        pretty_html = soup.prettify()
        options = {
            'page-size': 'A4',
            'margin-top': '0.75in',
            'margin-right': '0.75in',
            'margin-bottom': '0.75in',
            'margin-left': '0.75in',
            'encoding': "UTF-8",
            'no-outline': None,
            "enable-local-file-access": ""
        }

        pdfkit.from_string(pretty_html, output_file, options=options)        
        print(f"PDF created successfully: {output_file}")

    except requests.RequestException as e:
        print(f"An error occurred while downloading the HTML: {e}")
    except Exception as e:
        print(f"An error occurred while creating the PDF: {e}")

In [77]:
for i in range(len(fin_keys)):
    company_name = get_name(fin_keys[i])
    if (len(company_name) <= 0):
        not_happen.append(company_names[i])
        print("Didnt happen for company - " + company_names[i])
    else:
        date = get_date(company_name)
        xo = company_name.split(' ')
        parsed_string = '%20'.join(xo)
        xoo = date.split(' ')
        parsed_date = '_'.join(xoo)
        if (len(date) > 0):
            try:
                url1 = 'https://connect.acuite.in/fcompany-details/' + parsed_string + '/' + parsed_date
                output_file = os.path.join("Acuite_Ratings", company_name, f"{company_name} - {date}.pdf")
                html_to_pdf(url1, output_file)
            except:
                not_happen.append(company_names[i])
                print("Didnt happen for company - " + company_names[i])
        else:
            not_happen.append(company_names[i])
            print("Didnt happen for company - " + company_names[i])

First date: 4th Apr 24
PDF created successfully: Acuite_Ratings\AMAR INFRASTRUCTURE LIMITED\AMAR INFRASTRUCTURE LIMITED - 4th Apr 24.pdf
No company name found in the JSON data.
Didnt happen for company - INSTINCT INFRA & POWER LIMITED
First date: 20th Mar 24
PDF created successfully: Acuite_Ratings\SRI SALASAR BALAJI AGRO TECH PRIVATE LIMITED\SRI SALASAR BALAJI AGRO TECH PRIVATE LIMITED - 20th Mar 24.pdf
First date: 12th Feb 24
PDF created successfully: Acuite_Ratings\PRADIP POLYFILS PRIVATE LIMITED\PRADIP POLYFILS PRIVATE LIMITED - 12th Feb 24.pdf
First date: 30th Apr 24
PDF created successfully: Acuite_Ratings\BATLIBOI ENVIRONMENTAL ENGINEERING LIMITED\BATLIBOI ENVIRONMENTAL ENGINEERING LIMITED - 30th Apr 24.pdf
First date: 9th Aug 23
PDF created successfully: Acuite_Ratings\SHREE GOPINATHJI MOTORS PRIVATE LIMITED\SHREE GOPINATHJI MOTORS PRIVATE LIMITED - 9th Aug 23.pdf
No company name found in the JSON data.
Didnt happen for company - SHRISTI COTSPINN PRIVATE LIMITED
No company name

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [1]:
len(not_happen)

NameError: name 'not_happen' is not defined